In [6]:
from llama_stack_client import LlamaStackClient

In [2]:
client = LlamaStackClient(base_url="http://localhost:8321")

models = client.models.list()

llm = next(m for m in models if m.model_type == "llm")
model_id = llm.identifier

print("Model:", model_id)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


Model: llama3.2:3b


In [3]:
response = client.inference.chat_completion(
    model_id=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about coding"},
    ],
)

print(response.completion_message.content)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/inference/chat-completion "HTTP/1.1 200 OK"


Here is a haiku about coding:

Lines of code unfold
Logic's gentle, secret dance
Beauty in the bits


In [18]:
client.toolgroups.unregister(toolgroup_id="mcp::docling")

INFO:httpx:HTTP Request: DELETE http://localhost:8321/v1/toolgroups/mcp::docling "HTTP/1.1 200 OK"


In [20]:
client.toolgroups.register(
    toolgroup_id="mcp::docling",
    provider_id="model-context-protocol",
    mcp_endpoint={"uri": "http://localhost:8000/sse"}
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


In [21]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha'),
 ToolGroup(identifier='mcp::docling', provider_id='model-context-protocol', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'), provider_resource_id='mcp::docling')]

In [23]:
from llama_stack_client import Agent
from llama_stack_client.types.agent_create_params import AgentConfig

agent_config = AgentConfig(
    model=model_id,
    instructions="""You are a helpful assistant with access to tools that can convert documents to markdown.
When asked to convert a document, use the 'convert_document' tool.
You can also extract tables with 'extract_tables' or get images with 'convert_document_with_images'.
Always use the appropriate tool when asked to process documents.""",
    toolgroups=["mcp::docling"],
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False,
    tool_choice="auto",
    tool_prompt_format="python_list",
    max_tool_calls=3,
)

In [24]:
agent = Agent(client, agent_config)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.392015 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.773963 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Adocling "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}